In [7]:
import pandas as pd
import re

# --- ▼▼▼ ここをあなたのCSVファイルのパスに書き換えてください ▼▼▼ ---
file_path = "/work/test/DIL/random.csv"
# --- ▲▲▲ ---------------------------------------------------- ▲▲▲ ---
def import_csv_file(file_path):
    """
    指定されたCSVファイルを読み込み、DataFrameを返す。
    ファイルが存在しない場合はエラーメッセージを表示する。
    """
    try:
        df = pd.read_csv(file_path)
        # 先に重複行を削除しておくのが安全
        df = df.drop_duplicates().reset_index(drop=True)
        print(f"✅ データファイル '{file_path}' を正常に読み込みました。")
        return df
    except FileNotFoundError:
        print(f"❌ エラー: ファイルが見つかりません。パスを確認してください: '{file_path}'")
        return None
    


In [8]:
def clean_column_names(df):
    # 整形後のデータを格納するためのリスト
    clean_data_list = []

    # 現在どのモデル（どのタスクまで訓練済みか）を評価しているかを追跡する変数
    current_train_task = -1
    # train_taskごとのテスト結果を追跡するカウンター
    test_counter = 0

    print("CSVの各行を解析し、訓練タスクとテストタスクを割り振ります...")

    # DataFrameの各行を順に処理
    for index, row in df.iterrows():
        
        # --- 訓練時のログか、テストのみのログかを判断 ---
        # 'モデル'列に値がある行を、新しい訓練が完了した行と見なす
        if pd.notna(row['モデル']):
            # チェックポイントパスから訓練タスク番号を抽出
            match = re.search(r'train_task_(\d+)', str(row['チェックポイントパス']))
            current_train_task = int(match.group(1)) if match else 0
            
            # この行は train.test=true による「自身のタスクでのテスト」結果
            # あとのループで処理するため、ここではテストカウンターのリセットのみ行う
            test_counter = 0
            
        else:
            # 'モデル'列が空の行は、Bashスクリプトの内側ループによるテスト結果
            if current_train_task != -1:
                # test_counterがテスト対象のタスク番号(j)に対応する
                test_task = test_counter
                
                result_row = {
                    '訓練完了タスク': current_train_task,
                    'テスト対象タスク': test_task,
                    'テスト精度 (Test Acc)': row['テスト精度 (Test Acc)']
                }
                clean_data_list.append(result_row)
                
                # 次のテスト行のためにカウンターを1増やす
                test_counter += 1

    # 訓練時の自身のタスクに対するテスト結果も、最後に別途追加する
    for index, row in df[df['モデル'].notna()].iterrows():
        match = re.search(r'train_task_(\d+)', str(row['チェックポイントパス']))
        train_task = int(match.group(1)) if match else 0
        clean_data_list.append({
            '訓練完了タスク': train_task,
            'テスト対象タスク': train_task, # 自身のタスク
            'テスト精度 (Test Acc)': row['テスト精度 (Test Acc)']
        })

    # リストから新しいDataFrameを作成し、重複を最終的に削除
    clean_df = pd.DataFrame(clean_data_list)
    clean_df = clean_df.drop_duplicates(subset=['訓練完了タスク', 'テスト対象タスク'], keep='last')
    print("✅ 割り振りが完了し、重複が削除されました。")

    # 訓練タスクごとにグループ化し、テストタスクでソート
    grouped = clean_df.groupby('訓練完了タスク')

    # 最終的な結果を格納するリスト
    final_accuracy_array = []

    # 訓練タスク0から順番に処理
    for train_task_num in sorted(grouped.groups.keys()):
        group = grouped.get_group(train_task_num)
        # テストタスクの昇順に並び替え
        sorted_group = group.sort_values(by='テスト対象タスク')
        # テスト精度の値だけをリストとして抽出
        accuracies = sorted_group['テスト精度 (Test Acc)'].tolist()
        final_accuracy_array.append(accuracies)
        
    print("\n--- 最終的なテスト精度配列 ---")
    # 綺麗に表示
    for i, accs in enumerate(final_accuracy_array):
        print(f"Task {i} 完了後: {accs}")

In [12]:
file_path="/work/test/DIL/er_random_5000.csv"
import_csv_file(file_path)

ParserError: Error tokenizing data. C error: Expected 24 fields in line 110, saw 25
